# Semantic Chunking 
## Overview
This code implements a semantic chunking approach for processing and retrieving information from PDF documents. Unlike traditional methods that split text based on fixed character or word counts, semantic chunking aims to create more meaningful and context-aware text segments.

## Motivation
Traditional text splitting methods often break documents at arbitrary points, potentially disrupting the flow of information and context. Semantic chunking addresses this issue by attempting to split text at more natural breakpoints, preserving semantic coherence within each chunk.

## Key Components
1-PDF processing and text extraction\
2-Semantic chunking using LangChain's SemanticChunker\
3-Vector store creation using FAISS and OpenAI embeddings\
4-Retriever setup for querying the processed documents\
## Method Details
### Document Preprocessing
1-The PDF is read and converted to a string using a custom read_pdf_to_string function.\


### Step 1: Import Libraries and Load OpenAI API Key
In this step, we import necessary libraries and load environment variables to access the OpenAI API. We append the parent directory to the Python path so we can access helper functions and evaluation modules.

We also introduce a new type of text splitter called `SemanticChunker`, which will later be used to chunk the text based on meaning rather than fixed size. Finally, the OpenAI API key is securely loaded using the `.env` file.
\
\
Description: This cell sets up the environment by importing necessary libraries for semantic chunking and evaluation. We use the SemanticChunker class to split text based on its meaning rather than fixed sizes. Additionally, we ensure that the OpenAI API key is loaded securely from the .env file for further use in the notebook.

###  Import Necessary Libraries
This cell imports all required libraries for handling Azure OpenAI, document embeddings, vector storage, document loading, and processing.

#### Explanation:
 This imports Azure OpenAI API, document embedding and retrieval libraries, FAISS for vector stores, and utilities for document processing and text splitting. PyMuPDF (fitz) is used for handling PDF files.

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.embeddings import Embeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import fitz  # PyMuPDF


In [3]:
load_dotenv('variables.env')
# Azure OpenAI configuration
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

In [ ]:
print(azure_openai_api_key)
print(azure_openai_api_version)
print(azure_endpoint)

###  Configure Azure OpenAI API
This cell sets up the Azure OpenAI client with your credentials (API key, endpoint, and API version).

Explanation: Initializes the Azure OpenAI client with the required API key, endpoint, and version. Make sure to replace these with your actual Azure credentials.

In [6]:
# TODO: Set up the Azure OpenAI client
# Hint: Use the AzureOpenAI class
azure_client = # Your code here

# Cell 5: Define Custom Embeddings Class for Azure OpenAI

####  Define Custom Embeddings Class for Azure OpenAI
This cell defines a custom class that interacts with Azure OpenAI to embed documents and queries using the embeddings model.

Explanation: This class handles document embeddings by calling Azure OpenAI's embedding model (text-embedding-ada-002). It embeds both queries and documents.

In [7]:
class CustomAzureEmbeddings(Embeddings):
    def __init__(self, client):
        self.client = client

    def embed_documents(self, texts):
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text):
        response = self.client.embeddings.create(
            model="text-embedding-ada-002",
            input=text
        )
        return response.data[0].embedding


####  Read PDF and Extract Text
This function extracts text from the PDF file using PyMuPDF.

Explanation: This function reads a PDF file and extracts its text content, which will be later used for chunking and embedding.

In [8]:
# Cell 6: Read PDF and Extract Text

def read_pdf_to_string(path):
    # TODO: Implement PDF text extraction using PyMuPDF (fitz)
    # Hint: Open the PDF, iterate through pages, and extract text
    doc = # Your code here
    content = ""
    # Your code here to extract text from each page
    return content

###  Semantic Text Splitting
This function splits large chunks of text into semantically meaningful sections using cosine similarity.

Explanation: This function splits the text into chunks based on semantic similarity. It uses sentence embeddings and compares them using cosine similarity to create meaningful chunks for further processing.

In [9]:
# Cell 7: Semantic Text Splitting

def semantic_split(text, embeddings_client, max_chunk_size=1000, similarity_threshold=0.7):
    # TODO: Split the text into sentences
    # Hint: Use a list comprehension with text.split('.') to get sentences, and strip whitespace
    sentences = # Your code here 

    # TODO: Generate embeddings for each sentence
    # Hint: Use the embed_documents method of the embeddings_client
    sentence_embeddings = # Your code here 
    
    chunks = []
    current_chunk = sentences[0]
    current_embedding = sentence_embeddings[0]
    
    # Your code here to create semantic chunks
    # Hint: Iterate through sentences and embeddings, comparing cosine similarity
    # Use np.mean to update current_embedding when adding to a chunk
    # Append to chunks when similarity is below threshold or max size is reached
    
    return chunks

###  Encode PDF and Create FAISS Index
This cell encodes the PDF, embeds the chunks, and creates a FAISS index.

Explanation: This function reads the PDF, splits it semantically, embeds the chunks using Azure OpenAI, and stores them in a FAISS vector store for efficient retrieval.

In [10]:
# Cell 8: Encode PDF and Create FAISS Index

def encode_pdf(path):
    try:
        embeddings_client = CustomAzureEmbeddings(azure_client)

        # TODO: Read PDF content
        # Hint: Use the read_pdf_to_string function you defined earlier
        content = # Your code here

        # TODO: Split content into semantic chunks
        # Hint: Use the semantic_split function with the content and embeddings_client
        chunks = # Your code here

        # TODO: Create Document objects from chunks
        # Hint: Use a list comprehension to create Document objects
        texts = # Your code here 

        # TODO: Generate embeddings for the chunks
        # Hint: Use embeddings_client.embed_documents() with a list comprehension
        embeddings = # Your code here

        # TODO: Convert embeddings to numpy array and print its shape
        embeddings_array = # Your code here
        print(f"Document embeddings shape: {embeddings_array.shape}")

        # TODO: Get the dimension of the embeddings
        # Hint: Use the shape attribute of the numpy array
        dimension = # Your code here 

        # TODO: Create FAISS index
        # Hint: Use FAISS.from_embeddings() with zipped texts and embeddings
        index = # Your code here FAISS.from_embeddings(zip([t.page_content for t in texts], embeddings), embeddings_client)

        print(f"FAISS index contains {len(index.docstore._dict)} documents")

        return index

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

#### Main Execution for Vector Store Creation
This cell handles the main execution for creating the vector store from the PDF.

Explanation: Reads the PDF and encodes it using the previously defined functions. This stores the document embeddings in a FAISS index.

In [ ]:
# Cell 9: Main Execution for Vector Store Creation

pdf_path = "./data/Understanding_Climate_Change.pdf"

# TODO: Create the vector store
# Hint: Use the encode_pdf function and handle exceptions
try:
    vectorstore = # Your code here
    print("Vector store created successfully.")
except Exception as e:
    print(f"An error occurred during vector store creation: {e}")

###  Retrieve Relevant Documents
This cell creates a retriever using the FAISS vector store and retrieves relevant documents based on a query.

Explanation: This part retrieves the most relevant documents from the FAISS index based on a user's query. It prints the context of the retrieved documents.

In [ ]:
# Cell 10: Retrieve Relevant Documents

# TODO: Create a retriever from the vector store
# Hint: Use the as_retriever() method
retriever = # Your code here

query = "What is the main cause of climate change?"

# TODO: Retrieve relevant documents
# Hint: Use the get_relevant_documents() method and join the results.use .join()
try:
    docs = # Your code here
    context = # Your code here
    print("\nRetrieved context:")
    print(context)
except Exception as e:
    print(f"An error occurred during document retrieval: {e}")



###  Generate an Answer Using Azure OpenAI GPT-4o
This cell uses Azure OpenAI GPT-4o to generate an answer to the user's query based on the retrieved context.

Explanation: This part generates an answer to the user's query by passing the retrieved context and the question to Azure OpenAI GPT-4o, which generates a response.

In [ ]:
# Cell 11: Generate an Answer Using Azure OpenAI GPT-4o

# TODO: Generate an answer using the chat model
# Hint: Use the azure_client object's chat.completions.create() method
# You'll need to specify:
# - The model to use (your GPT-4o deployment name)
# - A list of messages, including a system message and a user message
# - The user message should include the context and query

try:
    response = # Your code here
    print("\nAnswer:")
    # TODO: Print the generated answer
    # Hint: The answer is in the 'content' of the first 'choice' message in the response
    print(# Your code here)
except Exception as e:
    print(f"An error occurred during answer generation: {e}")
